# GAN 基础学习

本笔记主要记录 GAN 这篇论文的原理和简要理解，由于学习 GAN 是为了给 DDPM 做铺垫，所以很多内容会比较简略，并且不涉及代码细节。

### 图像生成的思路

我们将图像视为一个服从概率分布的随机变量，那么我们学习的目标就是模拟这个分布。其他论文采取的是直接建模分布的方法，而本论文给出的思路是在“外部”创建一个生成器，通过调整生成器的参数使得生成的图像（或者说生成的数据）与原数据的分布尽可能接近。

### 对抗网络
本论文通过 **生成器（G）** 和 **判别器（D）** 两个网络相互博弈，训练 G 能够逼近真实的概率分布，训练 D 能够准确判断样本的真假。最终达到纳什均衡:
$$
\min_G \max_D V(D,G) = \mathbb{E}_{x \sim p_{data}(x)}[logD(x)] + \mathbb{E}_{z \sim p_z(z)}[log(1-D(G(z)))]
$$
该公式的意义是，定义一个价值函数（最大化D，最小化G），当达到纳什均衡时，$D(y) \equiv \frac{1}{2}$，此时 $V(D,G) = -log4$，表明生成器能够生成与真实样本一样的样本。这个价值函数相当于“损失函数”，指导参数更新的方向。

### 算法：小批量随机梯度优化
首先，定义超参数 $k$ 表示训练过程中D的迭代次数（通常增大k来提升判别能力）。$G(z;\theta _g)$ 是一个 MLP，$z$ 是随机数据，$\theta _g$ 是 MLP 的参数。$D(x;\theta _d)$ 是一个 MLP，$x$ 是真实数据，$\theta _d$ 是 MLP 的参数。

1. 进行小批量取样。
2. （这里迭代k次）训练 D，使 D “真为真”的概率最大化，“以假乱真”的概率最小化。
3. 训练 G，使 D “以假乱真”的概率最大化。

### 缺点
- 不收敛
- 没有损失函数，如果在博弈过程中出现“欺骗”行为，就会导致模式崩溃。这就是相比传统损失函数的的缺陷。